![image.png](attachment:3aed5e74-7b0c-4d8d-bc3c-bf92b6aceec3.png)

## Using the Power of NLP(BERT Embeddings and Annoy) to Recommend Simmilar Products to overcome the Problem of Cold-Start Problem
** Cold Start Problem: This constitutes a problem mainly for collaborative filtering algorithms due to the fact that they rely on the item's interactions to make recommendations.for that case we can leverage the capability of NLP to recommend the more simmilar products for the new user as there is no interaction with any of our products or byuing patterns of the user in the platform due to this to make more reliable recommendation we can use this.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import all libraries
import tensorflow as tf
from tensorflow import keras
import pandas as pd
# from keras.utils import plot_model
# from keras.utils.vis_utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pickle

from keras import backend as K

# from keras.layers.merge import add
import tensorflow as tf
import zipfile
# from keras.utils import np_utils
import transformers
import torch
# warnings.filterwarnings("ignore")
# warnings.simplefilter('ignore')
%matplotlib inline

In [3]:
df_products = pd.read_csv('products.csv')

df_products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
df_products.shape

(49688, 4)

In [5]:
#bert large uncased pretrained tokenizer
pretrained_weights = 'distilbert-base-uncased'
# Load pretrained model/tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [6]:
tokenized = df_products.product_name.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
tokenized

0                           [101, 7967, 11642, 16324, 102]
1                       [101, 2035, 1011, 3692, 5474, 102]
2        [101, 15873, 3585, 4895, 26760, 15558, 6675, 1...
3        [101, 6047, 3924, 4438, 20672, 7163, 17557, 26...
4                     [101, 2665, 7029, 15933, 12901, 102]
                               ...                        
49683    [101, 21092, 1010, 6420, 4487, 16643, 11001, 1...
49684    [101, 4372, 13675, 5833, 2063, 25043, 14015, 2...
49685           [101, 2396, 29196, 4524, 23361, 2618, 102]
49686    [101, 6047, 3468, 4859, 7965, 18089, 4318, 493...
49687            [101, 4840, 17952, 2075, 4550, 8043, 102]
Name: product_name, Length: 49688, dtype: object

In [8]:
dict(zip(tokenizer.all_special_tokens,tokenizer.all_special_ids))

{'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}

In [9]:
print(tokenized[0:2])

0        [101, 7967, 11642, 16324, 102]
1    [101, 2035, 1011, 3692, 5474, 102]
Name: product_name, dtype: object


In [10]:
#Later we will only average those tokens embeddings
text_len = [len(v) for v in tokenized]
text_len[:5]

[5, 6, 12, 14, 6]

In [11]:
#limit the maxlen to 20
maxlen = 20
for ix,token in enumerate(tokenized):
    if len(token) >= maxlen:
        token = token[:maxlen]
    else:
        token = token + [0] * (maxlen-len(token))
    tokenized[ix] = list(token)

In [12]:
tokenized = np.array(list(tokenized))
tokenized.shape

(49688, 20)

In [13]:
print(tokenized[0:2])

[[  101  7967 11642 16324   102     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  101  2035  1011  3692  5474   102     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]]


In [14]:
bert_model = transformers.TFDistilBertModel.from_pretrained(pretrained_weights)
bert_model.summary()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "tf_distil_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
Total params: 66362880 (253.15 MB)
Trainable params: 66362880 (253.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
# Creatinga hashmap and map the productId to each their product id's
product_ids = list(df_products.product_id)
product_name = list(df_products.product_name)

product_map = {}
for i in range(len(product_ids)):
    product_map[product_ids[i]] = product_name[i]

In [16]:
len(set(product_ids)),len(set(product_name))

(49688, 49688)

In [17]:
product_map[1]

'Chocolate Sandwich Cookies'

In [18]:
product_ids[:5], product_name[:5]

([1, 2, 3, 4, 5],
 ['Chocolate Sandwich Cookies',
  'All-Seasons Salt',
  'Robust Golden Unsweetened Oolong Tea',
  'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce',
  'Green Chile Anytime Sauce'])

In [19]:
def compress_tokens_embedding(arr,k,step):
    output = []
    r = arr.shape[0] # index No of product descriptions
    c = arr.shape[2] # Embedding size which is of 768 dimension
    for i in range(r):
        tmp = [product_ids[k+i],product_name[k+i]]
        x = np.mean( np.array(arr[i])[:text_len[k+i],:], axis = 0)
        for j in range(0,c,step): #Compression
            tmp.append(np.mean(x[j:j+step]))
        output.append(tmp)
    return np.array(output)

* Since we have 49.6K products & 49.6k * 768 dimension embedding becomes a lot to keep in memory, we process in batches.

* In first approach we will use the obtained hidden vector representation of all the tokens and do average pooling to get 768-dimension embedding ignoring the padded tokens.

* We will then compress the 768-dimension embedding to 64-dimensional by pooling across dimensions.

In [20]:
#as we run our model to generate the embeddings of products into batch of 80 so this is no of files we can get
49688/80

621.1

In [21]:
input_ids[0:0+1]

NameError: ignored

In [ ]:
last_hidden_states = bert_model(input_ids[0:0+1])[0][:,1:maxlen+1,:]

In [ ]:
#last_hidden_states comprised of 1st products description into 768 vectors dimension which we performed man pooling
t=np.array(last_hidden_states)

In [ ]:
t.shape

(1, 19, 768)

In [22]:
# code to store the product description in batch of 80 and store in 49688//80 = 621 different files and store the file using mean_avg_pooling technique
input_ids = tokenized
embedding_size = 768
step = 80
k = 0
avg_bits = 12
file_counter = 0
compressed_embedding_size = embedding_size//avg_bits  #64-dimensional embedding
# with strategy.scope():
while k < input_ids.shape[0]:
    last_hidden_states = bert_model(input_ids[k:k+step])[0][:,1:maxlen+1,:] #First token to 20th (max-len) token
    output = compress_tokens_embedding(np.array(last_hidden_states),k,avg_bits)
    col = ['product_id','product_name'] + [str(v) for v in range(compressed_embedding_size)]
    df_output = pd.DataFrame(output,columns = col)
    df_output.to_csv('file_' + str(file_counter) + '.csv', header = True, index=False)
    file_counter += 1
    k += step

In [23]:
!ls file*.csv | wc


    622     622    7976


In [24]:

!ls file*.csv

file_0.csv    file_180.csv  file_260.csv  file_340.csv	file_420.csv  file_500.csv  file_581.csv
file_100.csv  file_181.csv  file_261.csv  file_341.csv	file_421.csv  file_501.csv  file_582.csv
file_101.csv  file_182.csv  file_262.csv  file_342.csv	file_422.csv  file_502.csv  file_583.csv
file_102.csv  file_183.csv  file_263.csv  file_343.csv	file_423.csv  file_503.csv  file_584.csv
file_103.csv  file_184.csv  file_264.csv  file_344.csv	file_424.csv  file_504.csv  file_585.csv
file_104.csv  file_185.csv  file_265.csv  file_345.csv	file_425.csv  file_505.csv  file_586.csv
file_105.csv  file_186.csv  file_266.csv  file_346.csv	file_426.csv  file_506.csv  file_587.csv
file_106.csv  file_187.csv  file_267.csv  file_347.csv	file_427.csv  file_507.csv  file_588.csv
file_107.csv  file_188.csv  file_268.csv  file_348.csv	file_428.csv  file_508.csv  file_589.csv
file_108.csv  file_189.csv  file_269.csv  file_349.csv	file_429.csv  file_509.csv  file_58.csv
file_109.csv  file_18.csv   file_26.csv  

In [26]:
chk=pd.read_csv('file_0.csv')
chk.shape

(80, 66)

In [27]:
# This is how abg_embedding of products look like
chk.head()

,product_id,product_name,0,1,2,3,4,5,6,7,...,54,55,56,57,58,59,60,61,62,63
0,1,Chocolate Sandwich Cookies,0.011488,0.022841,0.015680,0.006920,0.002362,-0.093257,0.054916,0.134000,...,-0.008471,-0.014356,0.001361,-0.097843,-0.017851,0.014280,-0.093869,-0.025736,0.173451,-0.077830
1,2,All-Seasons Salt,-0.063548,0.105239,0.004789,-0.081728,0.015962,-0.051017,-0.051524,0.105884,...,0.032405,-0.005216,-0.016199,-0.076851,0.012476,0.071005,-0.030361,-0.030365,0.149966,-0.029466
2,3,Robust Golden Unsweetened Oolong Tea,0.019069,0.059295,0.006866,-0.050576,-0.015106,-0.038400,0.021967,0.054963,...,0.098647,-0.010491,-0.016024,-0.051417,-0.010944,0.032382,-0.048911,0.033894,0.097964,-0.108573
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,0.042045,0.055191,0.013291,0.019555,-0.055852,-0.073094,0.070101,0.087900,...,0.098411,-0.035836,-0.018626,-0.033904,-0.016163,-0.001881,-0.086864,-0.037667,0.106230,-0.102190
4,5,Green Chile Anytime Sauce,0.049123,0.035920,0.046164,0.036068,-0.009008,-0.053841,0.018368,0.156355,...,-0.016332,-0.006348,-0.020335,-0.064621,-0.003227,0.032786,-0.089255,-0.023770,0.146606,-0.050155


### Using annoy package we are trying to find the top K most nearest neighbour by leveraging the Products Embeddings
*** Below is the function to generate the Most Relatable Products(nearest, simmilar products) that we can use for recommendation in our Final Inference Pipeline**


In [29]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=550735 sha256=0aa92b56781acb82272baf4396ec67d87ba8fc6996c288580c59e2e8cdffd10e
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy


In [30]:
from annoy import AnnoyIndex
def create_index_find_similar_items(files):
    dfs = []
    for i in range(files):
        dfs.append(pd.read_csv('file_'+ str(i)+'.csv'))
    df_embedding = pd.concat(dfs)
    dfs.append(pd.read_csv('file_'+ str(i)+'.csv'))
    embedding_size = compressed_embedding_size
    a = AnnoyIndex(embedding_size, 'euclidean')
    for ix,row in df_embedding.iterrows():
        key = int(row['product_id'])
        vec = list(row[[str(v) for v in range(compressed_embedding_size)]])
        a.add_item(key,vec)

    a.build(100) # 100 trees
    a.save('test.tree')
    u = AnnoyIndex(embedding_size,  'euclidean')
    u.load('test.tree')

    top_k = 20
    mat = []
    for ix,row in df_embedding.iterrows():
        item = int(row['product_id'])
        mat.append([item] + u.get_nns_by_item(item, top_k+1)[1:])

#     print(len(mat),len(mat[0]))

    cols = ['product_id']
    for i in range(top_k):
        cols += ['nearest_{}'.format(i+1)]
    print(cols)

    df_neighbors1 = pd.DataFrame(mat, columns = cols)

    df_neighbors2 = df_neighbors1.copy()

    for c in cols:
        df_neighbors2[c] = df_neighbors2[c].apply(lambda v : product_map[v])

    return df_neighbors2


In [31]:
df_neighbors = create_index_find_similar_items(files= 622)

['product_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


In [32]:
df_neighbors.head(10)

,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,Chocolate Sandwich Cookies,Chocolate Chunk Cookies,Chocolate Butter Cookies,Chocolate Breakfast Bites,Milano Double Chocolate Cookies,Chocolate Chunk Cookie,Peanut Butter Cookies,Peanut Chocolate Candy,Milano Milk Chocolate Cookies,Chocolate Snack Grahams,...,Tray Cookies Chocolate Chip,Lemon Butter Cookies,Mini Chocolate Chunk Cookies,Lemon Twist Sandwich Cookies,Chocolate Peanut Butter,Chocolate Breakfast Biscuits Chocolate,Peanut Butter Breakfast Squares,Milano Orange Chocolate Cookies,Chocolate Chip Cookies,Glazed Chocolate Pie
1,All-Seasons Salt,All-Bran Buds Cereal,Sour Gold-Bears,All-Purpose Flour,New Mama Bottom Spray,Season All Seasoned Salt,In-Wash Scent Booster,Light Thousand Island Dressing,Green Tea K-Cups,Safe-Cut Can Opener,...,Whole Holland-Style onions,Basil Stir-In Paste,Eye Liner Black,Lip Shimmer - Fig,Cake Mate Decors Blue,Black Mission Figs,Soft-Picks,Supreme Matcha Green,Lip Shimmer - Plum,Rose Day Cream
2,Robust Golden Unsweetened Oolong Tea,Unsweetened Golden Oolong Tea,Unsweetened Oolong Tea Shot,Original Iced Matcha Unsweetened Tea,White Tea Unsweetened Hint O'Mint,Unsweetened Organic Ginger Oasis Tea,Organic Unsweetened OriginalWhite Tea,Frozen Unsweetened Whole Strawberries,Unsweetened Pure & Smooth Green Tea,Organic Unsweetened Black Tea,...,Unsweetened Elegant Jasmine Green Tea,Unsweetened Lemongrass Green Tea,Unsweetened All Natural Black Coffee,Unsweetened Lemon Flavor Real Brewed Tea,Organic Unsweetened Berry Coconut Granola,Unsweetened Natural Big Cup Apple Sauce,Pure Leaf Unsweetened Real Brewed Tea,Unsweetened Mint Lemon Mate,Organic Unsweetened Lemon Iced Green Tea,Organic Unsweetened Creamy Cashewmilk
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,Smart Ones Classic Favorites Thin Crust Pepper...,Sweet & Smoky Sloppy Joes Skillet Sauce,Chickpea & Tomato with Beef Stage 3 Baby Food,Hot Taco Lawry's Spices & Seasonings,Signature Ripe Tomato Extra Hot Gourmet Salsa,Grapefruit Bergamot Shea Butter Body Wash,Natural California Raisins Mini Snack Boxes,Nutrish® Real Chicken & Veggies Super Premium ...,Smart Ones Tasty American Favorites Tuna Noodl...,...,Crunchy Oats 'n Honey Granola Bars,Homestyle Thick & Hearty Traditional Pasta Sauce,Triscuit Whole Foods Brown Rice Tomato & Sweet...,Issimo Latte Macchiato Iced Coffee Drink with ...,Naturally Rising Crust Canadian Style Bacon & ...,All Natural Chickpea Snacks! Smoky Chili & Lime,Danimals Strawberry Explosion Flavored Smoothie,Kids! Chewables Probiotic Tablets Natural Burs...,Cool Ranch Flavored Tortilla Chips,Broccoli Cauliflower & Carrot Ranch Crunch Sna...
4,Green Chile Anytime Sauce,Mushroom Italian Sauce,Honey Barbecue Sauce,Roasted Mole Sauce,Rustic Tomato Vodka Sauce,Alla Vodka Sauce,Light Creamy Alfredo Sauce,Mole Mexican Cooking Sauce,Hot Insanity Sauce,Butter Cooking Spray,...,Tomato Sauce Classic,Raw Carrot Soup,Southwest Chili Beans,Select Soy Sauce,Classic 1837 Steak Sauce,Sesame Cooking Sauce,Chile Verde Chicken,Tomato Basil Soup,Asian Chopped Salad,Green Curry Paste
5,Dry Nose Oil,Nasal Cleansing Pot,Dry Blackberry Soda,Ear Drying Drops,Walnut Spray Oil,Rich Chili Chips,Saline Nasal Mist,Perfect Pinch Vegetable Supreme,Cool Mint Gum,Scotch Packing Tape,...,Dry Erase Markers,Rose Essential Oil,Pumpkin Seed Oil,Rose Absolute Oil,Sweet Kernel Corn,Mild Pepper Rings,Butter Cooking Spray,Lightly Dried Basil,Grapeseed Spray Oil,Zero Cake Batter
6,Pure Coconut Water With Orange,Pure Coconut Water With Pulp,Pure Coconut Water with Tropical Fruit,Coconut Water with Orange Mango,Pure Organic Coconut Water,Pure Coconut Water with Pineapple,Coconut Water with Pulp,Coconut Juice with Lime,"Coconut Water, Pure Organic",Organic Pure Extra Virgin Coconut Oil,...,Natural Pulp Free Coconut Water,Organic Coconut Water With Pinea

In [33]:
df_neighbors.to_csv('neighbors_token_embedding_avg_compressed.csv', index=False, header=True)

In [34]:
!rm file*.csv

In [35]:
#Will compress 768 bits embedding to 768/42 = 32 bits
def compress_cls_embedding(arr,k,step):
    output = []
    r = arr.shape[0] # No of product descriptions
    c = arr.shape[2] # Embedding size
    for i in range(r):
        tmp = [product_ids[k+i],product_name[k+i]]
        x = np.array(list(arr[i,0,:]))
        for j in range(0,c,step):
            tmp.append(np.mean(x[j:j+step]))
        output.append(tmp)
    return np.array(output)

### 2nd Method just take the Class Embedding of 768 Dimension Not the input text tokens and compressed it into 64 dimension

In [36]:
input_ids = tokenized
embedding_size = 768
step = 80
k = 0
avg_bits = 12
file_counter = 0
compressed_embedding_size = embedding_size//avg_bits
# with strategy.scope():
while k < input_ids.shape[0]:
    last_hidden_states = bert_model(input_ids[k:k+step])[0][:,:1,:]
    output = compress_cls_embedding(np.array(last_hidden_states),k,avg_bits)
    col = ['product_id','product_name'] + [str(v) for v in range(compressed_embedding_size)]
    df_output = pd.DataFrame(output,columns = col)
    df_output.to_csv('file_' + str(file_counter) + '.csv', header = True, index=False)
    file_counter += 1
    k += step

In [37]:
!ls file*.csv | wc

!ls file*.csv

    622     622    7976
file_0.csv    file_180.csv  file_260.csv  file_340.csv	file_420.csv  file_500.csv  file_581.csv
file_100.csv  file_181.csv  file_261.csv  file_341.csv	file_421.csv  file_501.csv  file_582.csv
file_101.csv  file_182.csv  file_262.csv  file_342.csv	file_422.csv  file_502.csv  file_583.csv
file_102.csv  file_183.csv  file_263.csv  file_343.csv	file_423.csv  file_503.csv  file_584.csv
file_103.csv  file_184.csv  file_264.csv  file_344.csv	file_424.csv  file_504.csv  file_585.csv
file_104.csv  file_185.csv  file_265.csv  file_345.csv	file_425.csv  file_505.csv  file_586.csv
file_105.csv  file_186.csv  file_266.csv  file_346.csv	file_426.csv  file_506.csv  file_587.csv
file_106.csv  file_187.csv  file_267.csv  file_347.csv	file_427.csv  file_507.csv  file_588.csv
file_107.csv  file_188.csv  file_268.csv  file_348.csv	file_428.csv  file_508.csv  file_589.csv
file_108.csv  file_189.csv  file_269.csv  file_349.csv	file_429.csv  file_509.csv  file_58.csv
file_109.csv  fil

In [38]:
df_neighbors_cls = create_index_find_similar_items(files= 622)
df_neighbors_cls.head(20)

['product_id', 'nearest_1', 'nearest_2', 'nearest_3', 'nearest_4', 'nearest_5', 'nearest_6', 'nearest_7', 'nearest_8', 'nearest_9', 'nearest_10', 'nearest_11', 'nearest_12', 'nearest_13', 'nearest_14', 'nearest_15', 'nearest_16', 'nearest_17', 'nearest_18', 'nearest_19', 'nearest_20']


,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
0,Chocolate Sandwich Cookies,Chocolate Butter Cookies,Chocolate Chunk Cookies,Chocolate Chip Cookies,Chocolate Cookies,Cinnamon Bun Sandwich Cookies,Mini Chocolate Chunk Cookies,Chocolate Chip Mini Cookies,Pure Chocolate Cookies,Lemon Butter Cookies,...,Chocolate Cupcakes,Carrot Cake Cookies,Chocolate Chunk Cookie Dough,Sugar Free Chocolate Chunk Cookies,Double Chocolate Mini Cakes,Chocolate Chip Cookie Dough,Chocolate Alphabet Cookies,Mini Chocolate Chip Cookies,Cheese Cracker Sandwiches,Milk Chocolate Chips
1,All-Seasons Salt,Red Jr. Shot Glass,"Bars, Blueberry","Pizza, Cheese & Pesto","Candle, Meadows & Rain","Ice Cream, Cookies & Cream","Mustard, Stout & Stoneground","Syrup, Maple Creme","Baking Mix, Brownie","Burrito, Steak & Cheese",...,Pancakes Blueberry - 18 CT,Green Tea K-Cups,Island Vanilla Roll-On Perfume,"Gum, Peppermint",Lip Shimmer - Plum,Bite-Size Honey Gold Potatoes,"Protein Shake, Cookies & Cream","Nut Bar, Chocolate Chunk",Temptations Cat Treats - Creamy Dairy,Lemon-Lime Fridge Pack Soda
2,Robust Golden Unsweetened Oolong Tea,Natural Hibiscus Full Leaf Loose Herb Tea,Natural Vanilla Miniature Marshmallows,Organic Echinacea Plus Elderberry Tea,Organic Mekong Flower Brown Jasmine Rice,Asian Ginseng Liquid Herbal Extract,Jasmine Petal Whole Leaf Green Tea,Berry Pomegranate Liquid Water Enhancer,All Natural Lavender Sage Deodorant,Pure Seedless Marion Blackberry Preserves,...,Garden Herb Triscuit Crackers,Loose Leaf Mild Yerba Mate Tea,Red Honeysuckle Nectar Scented Candle,Vanilla Mint Moisturizing Conditioner,Pure Seedless Pacific Mountain Strawberry Pres...,All Natural Raspberry Sorbet,Blue Eucalyptus & Lavender Natural Laundry Det...,California Seedless Raisins,Organic Mixed Berry Thirst Quencher,Echinacea & Elderberry Organic Herbal Tea
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,Smart Ones Tasty American Favorites Tuna Noodl...,Smart Ones Classic Favorites Chicken Enchilada...,Baby Romaine Lettuce Leaf Mix Salad Clamshell,Yokids Organic Strawberry Banana Lowfat Yogurt...,Petite Creme Lowfat Blueberry Yogurt,Sparkling Limeade with Strawberry & Lychee Vie...,Clif Kid ZBar Organic Iced Lemon Cookie Energy...,PRO Alphonso Mango Lassi Yogurt Drink,Strawberry Banana Peach Light & Fit Nonfat Yogurt,...,YoKids Squeeze Organic Blueberry Blue Yogurt,Refreshers Strawberry Lemonade Sparkling Green...,Original Strawberry/Harvest Peach Variety Pack...,Light & Fit Greek Crunch Yogurt Peanut Butter ...,Original Strawberry/Strawberry Banana Variety ...,Organic Petite Creme Mon Cherry Amour Yogurt,Berry Lemonade with Chia Chewy Granola Bars,Zero Calorie Strawberry Lemonade Sparkling Wat...,Smart Ones Creamy Rigatoni With Broccoli & Chi...,Cherry Vanilla Creme Zero Calorie Stevia Soda
4,Green Chile Anytime Sauce,Rustic Tomato Vodka Sauce,Hot Insanity Sauce,Mild Taco Sauce,Zesty Cocktail Sauce,Honey Barbecue Sauce,Tomato Basil Sauce,La Vodka Cooking Sauce,Mild Barbecue Sauce,Apple Strawberry Sauce,...,La Vodka Pasta Sauce,Apple Peach Sauce,Tomato Cream Sauce,Roasted Garlic Pasta Sauce,Peppercorn Sauce,Sweet Honey Barbecue Sauce,Mild Korma Simmer Sauce,Orange Chicken Sauce,Yogurt Mint Sauce,Tomato Sauce
5,Dry Nose Oil,Mild Baby Oil,Garlic Oil,Facial Oil,Beef Broth Concentrate,Fish Oil,Chicken Broth Liquid Concentrate,Organic Veggie Ground,Canola Oil,Butter Cooking Spray,...,Chunk Light in Oil Tuna,Nasal Cleansing Salt Packets,Lightly Salted Peanuts,Mild Curry Paste,Organic Silken Tofu,Wheat Sandwich Thins,Vegetable Oil,Lightly Salted Peanut,Sweetened Condensed Milk,Organic Broth Ramen
6,Pure Coconut Water With Orange,Pure Coconut Water With Pulp,Pure Coconut Water with Pineapple,Pure Coconut Water,Organic Dark Cacao Raw Coconut Water,Pure Coconut Water with Tropical Fruit,Pure Organic Coconut Water,Organic Co

In [43]:
df_neighbors_cls[df_neighbors_cls["product_id"] == "Organic Hass Avocado"]

,product_id,nearest_1,nearest_2,nearest_3,nearest_4,nearest_5,nearest_6,nearest_7,nearest_8,nearest_9,...,nearest_11,nearest_12,nearest_13,nearest_14,nearest_15,nearest_16,nearest_17,nearest_18,nearest_19,nearest_20
47208,Organic Hass Avocado,Organic Hass Avocado Bag,Organic Dill Relish,Organic Emmer Farro,Organic Whole O's Cereal,Organic Jumbo Garlic,Organic Sweet Relish,Avocado Spread Original,Organic Whole Carrots,Organic Pummelo,...,Citrus Electrolyte Tabs,Organic Allspice,Organic Acai Juice,Organic Mint Bunch,Medium Suprema Salsa,Lip Calm Lip Balm,Organic Oranges Bag,Organic Carrot Juice,Hass Avocados,Wheat Sandwich Thins


In [ ]:
df_neighbors_cls.to_csv('neighbors_cls_embedding_compressed.csv', index=False, header=True)

## Thank You! Please Upvote! If you like my Work